# 환경에서 작업


스크립트를 Azure Machine Learning 작업으로 실행하는 경우 작업 실행에 대한 실행 컨텍스트를 정의해야 합니다. 한 가지 주요 구성은 스크립트가 실행될 컴퓨팅 대상입니다. 로컬 워크스테이션(이 경우 컴퓨팅 인스턴스) 또는 주문형으로 프로비전되는 Azure Machine Learning 관리형 컴퓨팅 클러스터와 같은 원격 컴퓨팅 대상일 수 있습니다.

이 Notebook에서는 컴퓨팅 클러스터를 만들고 작업에 대한 컴퓨팅 대상을 살펴봅니다.

## 시작하기 전에

이 Notebook에서 코드를 실행하려면  **최신 버전의 azureml-ai-ml** 패키지가 필요합니다. 아래 셀의 명령을 실행하여 이 패키지가 설치되어 있는지 확인합니다.

> **고**:
> **azure-ai-ml** 패키지가 설치되지 않은 경우 를 실행 `pip install azure-ai-ml` 하여 설치합니다.

In [ ]:
## 작업 영역에 연결

필요한 SDK 패키지를 설치했으므로 작업 영역에 연결할 수 있습니다.

작업 영역에 연결하려면 구독 ID, 리소스 그룹 이름 및 작업 영역 이름 등 식별자 매개 변수가 필요합니다. 리소스 그룹 이름 및 작업 영역 이름이 이미 채워져 있습니다. 명령을 완료하려면 구독 ID만 필요합니다.

필요한 매개 변수를 찾으려면 Studio의 오른쪽 위에 있는 구독 및 작업 영역 이름을 클릭합니다. 오른쪽에 창이 열립니다.

<p style="color:red;font-size:120%;background-color:yellow;font-weight:bold"> 구독 ID를 복사하고 **YOUR-SUBSCRIPTION-ID** 를 복사한 값으로 바꿉니다. </p>

## 스크립트를 작업으로 실행

모델을 학습하려면 먼저 **src** 폴더에 **diabetes_training.py** 스크립트를 만듭니다. 스크립트는 학습 데이터와 동일한 폴더에 있는 **diabetes.csv** 파일을 사용합니다.

스크립트의 시작 부분에 라이브러리를 가져옵니다. 이러한 라이브러리의 함수는 데이터를 처리하고 모델을 학습시키는 데 사용됩니다. 스크립트를 실행하는 데 사용하는 컴퓨팅에는 이러한 라이브러리가 설치되어 있어야 합니다.

In [ ]:
스크립트를 만든 후 스크립트를 작업으로 실행할 수 있습니다. 스크립트는 공통 라이브러리를 사용합니다. 따라서 pandas, numpy 및 scikit-learn을 포함하는 큐레이팅된 환경을 사용할 수 있습니다.

작업은 큐레이팅된 환경 `AzureML-sklearn-0.24-ubuntu18.04-py37-cpu`의 최신 버전인 를 사용합니다.

In [ ]:
작업이 실행되는 동안 이미 다음 셀을 실행할 수 있습니다.

## 환경 나열

작업 영역 내의 환경을 살펴보겠습니다. 

이전 작업에서는 큐레이팅된 환경 중 하나를 사용했습니다. 작업 영역에 이미 있는 모든 환경을 탐색하려면 환경을 나열할 수 있습니다. 

In [ ]:
큐레이팅된 모든 환경에는 **AzureML-** (사용자 고유의 환경에 이 접두사를 사용할 수 없음)를 시작하는 이름이 있습니다.

특정 환경을 검토하려면 해당 이름과 버전별로 환경을 검색할 수 있습니다. 예를 들어 이전 작업에 사용한 큐레이팅된 환경의 *설명* 및 *태그* 를 검색할 수 있습니다.

## 사용자 지정 환경 만들기 및 사용

큐레이팅된 환경에 스크립트를 실행하는 데 필요한 모든 Python 패키지가 포함되지 않은 경우 사용자 지정 환경을 직접 만들 수 있습니다. 환경에 필요한 모든 패키지를 나열하면 스크립트를 쉽게 다시 실행할 수 있습니다. 모든 종속성은 사용하는 컴퓨팅과 관계없이 작업 구성에서 지정할 수 있는 환경에 저장됩니다.

예를 들어 Docker 이미지에서만 환경을 만들 수 있습니다. PyTorch와 같은 특정 프레임워크에는 필요한 모든 항목이 이미 포함된 공용 Docker 이미지가 있습니다. 

Docker 이미지에서 환경을 만들어 보겠습니다.

In [ ]:
이제 환경이 작업 영역에 등록되었으며 스크립트를 작업으로 실행할 때 참조할 수 있습니다.

<p style="color:red;font-size:120%;background-color:yellow;font-weight:bold"> 작업이 빠르게 실패합니다! 오류 메시지를 검토합니다. </p>

오류 메시지는 pandas라는 모듈이 없다는 것을 알려줍니다. 이러한 오류에는 두 가지 가능한 원인이 있습니다.

- 스크립트는 pandas를 사용하지만 라이브러리(`import pandas as pd`)를 가져오지 않았습니다. 
- 스크립트는 스크립트 맨 위에 있는 라이브러리를 가져오지만 컴퓨팅에 라이브러리가 설치되지 않았습니다(`pip install pandas`).

스크립트를 검토한 `diabetes-training.py` 후 스크립트가 올바른지 확인할 수 있습니다. 즉, 라이브러리가 설치되지 않았습니다. 즉, 환경에 필요한 패키지가 포함되지 않았습니다.

이전 작업에 사용된 기본 Docker 이미지를 사용하여 새 환경을 만들어 보겠습니다. 이제 필요한 패키지가 설치되도록 conda 사양을 추가합니다. 먼저 다음 셀을 실행하여 conda 사양 파일을 만듭니다.

스크립트가 성공적으로 실행되도록 필요한 모든 종속성이 conda 사양 파일에 포함되어 있습니다.

기본 Docker 이미지 **와** conda 사양 파일을 사용하여 새 환경을 만들어 필요한 종속성을 추가합니다. Azure Machine Learning은 제공한 Docker 이미지 위에 conda 환경을 빌드합니다. 

In [ ]:
이제 새 환경으로 작업을 제출하여 스크립트를 실행할 수 있습니다.

새 사용자 지정 환경으로 작업을 제출하면 환경 빌드가 트리거됩니다. 새로 만든 환경을 처음 사용할 때 환경을 빌드하는 데 10~15분이 걸릴 수 있습니다. 즉, 작업을 완료하는 데 시간이 더 오래 걸립니다. 

작업을 제출하기 전에 환경 빌드를 수동으로 트리거하도록 선택할 수도 있습니다. 환경을 처음 사용할 때만 환경을 빌드해야 합니다. 

작업이 새 환경의 빌드를 트리거하는 경우 작업의 **출력 + 로그** 탭에서 빌드 로그를 검토할 수 있습니다. **azureml-logs/20_image_build_log.txt** 열어 환경 빌드의 로그를 검사합니다. 

![스크린샷 빌드 로그](./images/screenshot-logs.png)

In [ ]:
env = ml_client.environments.get("AzureML-sklearn-0.24-ubuntu18.04-py37-cpu", version=44)
print(env. description, env.tags)

## Create and use a custom environment

If a curated environment doesn't include all the Python packages you need to run your script, you can create your own custom environment. By listing all necessary packages in an environment, you can easily re-run your scripts. All the dependencies are stored in the environment which you can then specify in the job configuration, independent of the compute you use.

For example, you can create an environment simply from a Docker image. Certain frameworks like PyTorch will have a public Docker image that already includes everything you need. 

Let's create an environment from a Docker image:

In [ ]:
from azure.ai.ml.entities import Environment

env_docker_image = Environment(
    image="mcr.microsoft.com/azureml/openmpi3.1.2-ubuntu18.04",
    name="docker-image-example",
    description="Environment created from a Docker image.",
)
ml_client.environments.create_or_update(env_docker_image)

The environment is now registered in your workspace and you can reference it when you run a script as a job:

In [ ]:
from azure.ai.ml import command

# configure job
job = command(
    code="./src",
    command="python diabetes-training.py",
    environment="docker-image-example:1",
    compute="aml-cluster",
    display_name="diabetes-train-custom-env",
    experiment_name="diabetes-training"
)

# submit job
returned_job = ml_client.create_or_update(job)
aml_url = returned_job.studio_url
print("Monitor your job at", aml_url)

<p style="color:red;font-size:120%;background-color:yellow;font-weight:bold"> The job will quickly fail! Review the error message. </p>

The error message will tell you that there is no module named pandas. There are two possible causes for such an error:

- The script uses pandas but didn't import the library (`import pandas as pd`). 
- The script does import the library at the top of the script but the compute didn't have the library installed (`pip install pandas`).

After reviewing the `diabetes-training.py` script you can observe the script is correct, which means the library wasn't installed. In other words, the environment didn't include the necessary packages.

Let's create a new environment, using the base Docker image used in the previous job. Now, you'll add a conda specification to ensure the necessary packages will be installed. First, run the following cell to create the conda specification file:

In [ ]:
%%writefile src/conda-env.yml
name: basic-env-cpu
channels:
  - conda-forge
dependencies:
  - python=3.7
  - scikit-learn
  - pandas
  - numpy
  - matplotlib

Note that all necessary dependencies are included in the conda specification file for the script to run successfully.

Create a new environment using the base Docker image **and** the conda specification file to add the necessary dependencies. Azure Machine Learning will build the conda environment on top of the Docker image you provided. 

In [ ]:
from azure.ai.ml.entities import Environment

env_docker_conda = Environment(
    image="mcr.microsoft.com/azureml/openmpi3.1.2-ubuntu18.04",
    conda_file="./src/conda-env.yml",
    name="docker-image-plus-conda-example",
    description="Environment created from a Docker image plus Conda environment.",
)
ml_client.environments.create_or_update(env_docker_conda)

Now, you can submit a job with the new environment to run the script:

In [ ]:
from azure.ai.ml import command

# configure job
job = command(
    code="./src",
    command="python diabetes-training.py",
    environment="docker-image-plus-conda-example:1",
    compute="aml-cluster",
    display_name="diabetes-train-custom-env",
    experiment_name="diabetes-training"
)

# submit job
returned_job = ml_client.create_or_update(job)
aml_url = returned_job.studio_url
print("Monitor your job at", aml_url)

Submitting the job with the new custom environment triggers the build of the environment. The first time you use a newly created environment, it can take 10-15 minutes to build the environment, which also means your job will take longer to complete. 

You can also choose to manually trigger the build of the environment before you submit a job. The environment only needs to be built the first time you use it. 

When the job triggers the build of a new environment, you can review the logs of the build in the **Outputs + logs** tab of the job. Open **azureml-logs/20_image_build_log.txt** to inspect the logs of the environment build. 

![Screenshot build logs](./images/screenshot-logs.png)